## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/zobai/Class/world_weather_analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,35.47,89,100,1.99,snow
1,1,Springbok,ZA,-29.66,17.89,76.53,30,0,5.50,clear sky
2,2,Sinnamary,GF,5.38,-52.95,75.88,90,80,11.27,moderate rain
3,3,Arraial Do Cabo,BR,-22.97,-42.03,75.20,91,100,1.90,overcast clouds
4,4,Nancha,CN,47.13,129.27,5.18,75,100,8.46,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
# Country column seems to have some null values
preferred_cities_df.count()

City_ID                157
City                   157
Country                152
Lat                    157
Lng                    157
Max Temp               157
Humidity               157
Cloudiness             157
Wind Speed             157
Current Description    157
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# referred to the following documentation to drop by specfic columns:
# https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
preferred_cities_df.dropna(subset = ["Country"], inplace = True)

# New DataFrame without empty rows
clean_df = preferred_cities_df

clean_df.count()

C:\Users\zobai\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                152
City                   152
Country                152
Lat                    152
Lng                    152
Max Temp               152
Humidity               152
Cloudiness             152
Wind Speed             152
Current Description    152
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Springbok,ZA,76.53,clear sky,-29.66,17.89,
2,Sinnamary,GF,75.88,moderate rain,5.38,-52.95,
3,Arraial Do Cabo,BR,75.20,overcast clouds,-22.97,-42.03,
8,Vaini,TO,78.96,broken clouds,-21.20,-175.20,
9,Atuona,PF,78.48,clear sky,-9.80,-139.03,
16,Rikitea,PF,78.06,overcast clouds,-23.12,-134.97,
20,Gizo,SB,82.94,overcast clouds,-8.10,156.84,
41,Butaritari,KI,82.36,light rain,3.07,172.79,
51,Natal,BR,80.82,few clouds,-5.79,-35.21,
60,Hithadhoo,MV,82.18,overcast clouds,-0.60,73.08,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
# Using Paris as my location
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Adding lat/long to location key for the params
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# Doesn't seem like there are any empty Hotel names
hotel_df.count()

City                   152
Country                152
Max Temp               152
Current Description    152
Lat                    152
Lng                    152
Hotel Name             152
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("C:/Users/zobai/Class/world_weather_analysis/Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Added hotel name, city name, country code, and the weather description with maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.
# 30.0 and 31.0 (lat/long) is the geographic center of the Earth
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))